In [77]:
%load_ext autoreload
import os
workDir = os.getcwd()
user = 'testID' #CERN user ID
institution = 'UCSB' #UCSB
print(workDir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/lbrennan/Documents/Research/Coding/FiducialsQC-master


In [78]:
%run utils/GetExcel.ipynb

#The sensor file can contain the fiducials of the sensor, but can also have the 
#component heights for sensor in location P1 or P2.
#Methods can run either way! 

#pcbfile = workDir + '/data/8.18-DryRun1Try3.xls'
#pcbfile = workDir + '/data/LDV3D12_protomodule.xls'
fiducialfile = workDir + '/data/GantryTrayFiducialData.xls'
##fiducialfile = workDir + '/data/8.4.22test.xls'
#sensorfile = workDir + '/data/sensorfiducialtrialrun.xls'
##sensorfile = workDir + '/data/8.4.22negyccw.xls'
sensorfile = workDir + '/data/8-22DataReFormattedLDV3Run5.xls'
surveyfile = workDir + '/data/8-22DataReFormattedLDV3Run5.xls'
#surveyfile = workDir + '/data/LDV3D12_protomodule.xls'


survey = GetExcel(surveyfile)
survey.read()
print(survey.df)

sensor = GetExcel(sensorfile)
sensor.read()
#print("This is sensor", sensor.df.tail())
print("Using this file:", sensorfile)

fiducial = GetExcel(fiducialfile)
fiducial.read()
#print(fiducial.df.head())

pcb = GetExcel(pcbfile)
pcb.read()
print("This is PCB:", pcb.df.head())

    Unnamed: 0 Unnamed: 1         Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
3          NaN   Routine1             mm  Ɵ°        NaN        NaN        NaN   
6          NaN        NaN            Feature  Attribute    Nominal     Actual   
7          NaN        NaN  P1SensorFiducial1          X    137.141    136.874   
8          NaN        NaN                NaN          Y    180.772    180.514   
11         NaN        NaN  P1SensorFiducial2          X    136.789    137.183   
12         NaN        NaN                NaN          Y    340.770    340.544   
15         NaN        NaN  P1SensorFiducial3          X     56.917     57.123   
16         NaN        NaN                NaN          Y    295.600    295.670   
19         NaN        NaN  P1SensorFiducial4          X    217.002    216.919   
20         NaN        NaN                NaN          Y    225.929    225.363   

    Unnamed: 6  Unnamed: 7 Unnamed: 8  
3          NaN         NaN        NaN  
6   Upper Tol.  Lower Tol.  

In [79]:
%run utils/ReadDataframe.ipynb
%autoreload 2

if sensor.exists:
    senDF = ReadDataframe(sensor.df, sensorfile)
    senDF.readComponentFiducials()
    senDF.readComponentHeights()

if fiducial.exists:
    fidDF = ReadDataframe(fiducial.df, fiducialfile)
    fidDF.readTrayFiducials()

if pcb.exists:
    pcbDF = ReadDataframe(pcb.df, pcbfile)
    pcbDF.readComponentFiducials()
    pcbDF.readComponentHeights()
    
if survey.exists:
    surveyDF = ReadDataframe(survey.df, surveyfile)
    surveyDF.readComponentHeights()

This is Actual: Unnamed: 5
[(136.874, 180.514), (137.183, 340.544), (57.123, 295.67), (216.919, 225.363)]
This is Actual: Unnamed: 5


In [80]:
%run utils/QualityControl.ipynb
%autoreload 2

if sensor.exists:
    sensorP1 = QualityControl(fidDF, senDF, 'P1', False)
    if len(senDF.pointsP1): sensorP1.runTransform()
    if len(senDF.heightP1): sensorP1.runHeights()
    sensorP1.rating()
    print(sensorP1)

#    sensorP2 = QualityControl(fidDF, senDF, 'P2', False)
#    if len(senDF.pointsP2): sensorP2.runTransform()
#    if len(senDF.heightP2): sensorP2.runHeights()
#    sensorP2.rating()
#    print(sensorP2)
    
if pcb.exists:
    pcbP1 = QualityControl(fidDF, pcbDF, 'P1', True)
    if len(pcbDF.pointsP1): pcbP1.runTransform()
    if len(pcbDF.heightP1): pcbP1.runHeights()
    pcbP1.rating()
    print(pcbP1)

    pcbP2 = QualityControl(fidDF, pcbDF, 'P2', True)
    if len(pcbDF.pointsP2): pcbP2.runTransform()
    if len(pcbDF.heightP2): pcbP2.runHeights()
    pcbP2.rating()
    print(pcbP2)
    
if survey.exists:
    surveyComp = QualityControl(fidDF, surveyDF, 'Survey', False, True)
    if len(surveyDF.heightP1): surveyComp.runHeights()

This is self.points: [[ 57.123 295.67 ]
 [136.874 180.514]
 [137.183 340.544]
 [216.919 225.363]]
[(0.0, 0.0), (0.001, 355.043)]
[(137.03, 260.598), (62.131, 260.471)]
This is theta: 0.0
This is self.position: [(137.03, 260.598), (62.131, 260.471)]
This is self.newpoints: [[-79.907  35.072]
 [ -0.156 -80.084]
 [  0.153  79.946]
 [ 79.889 -35.235]]
This is self.newpoints[0][1] 35.072
This is self.newpoints[1][1] -80.084
This is self.newpoints[0][0] -79.90700000000001
This is self.newpoints[1][0] -0.1560000000000059
[-0.00525 -0.07525]
This is self.deg: [124.70444073793378, 124.69357561753583]
The module/component in P1 has completed QC!
Rotation --> Left Fiducial: 124.704 degrees, Right Fiducial: 124.694 degrees
The angle of the sensor in position P1 is RED. 
Placement --> (x,y) = (-0.005, -0.075) mm
The placement of the sensor in position P1 is YELLOW. 

The FINAL rating of the module in position P1 is RED.

[(0.0, 0.0), (0.001, 355.043)]
[(137.03, 260.598), (62.131, 260.471)]
This is 

In [81]:
%run utils/CreateXML.ipynb
%autoreload 2

if sensor.exists:
    if sensorP1.finalRating != 'NO INFO':
        sensorP1XML = CreateXML(sensorP1, user, institution)
        sensorP1XML.save(senDF.dir, senDF.name + '_P1.xml')

    if sensorP2.finalRating != 'NO INFO':
        sensorP2XML = CreateXML(sensorP2, user, institution)
        sensorP2XML.save(senDF.dir, senDF.name + '_P2.xml')
    
if pcb.exists:
    if pcbP1.finalRating != 'NO INFO':
        pcbP1XML = CreateXML(pcbP1, user, institution)
        pcbP1XML.save(pcbDF.dir, pcbDF.name + '_P1.xml')
    
    if pcbP2.finalRating != 'NO INFO':
        pcbP2XML = CreateXML(pcbP2, user, institution)
        pcbP2XML.save(pcbDF.dir, pcbDF.name + '_P2.xml')

Saving UPLOAD XML file to  /Users/lbrennan/Documents/Research/Coding/FiducialsQC-master/data/8-22DataReFormattedLDV3Run5_P1_upload.xml


NameError: name 'sensorP2' is not defined

In [ ]:
%run utils/HeightPlotter.ipynb
%autoreload 2

if sensor.exists:
    if len(sensorP1.hgts):
        label = senDF.name + ' P1'
        senPlotP1 = HeightPlotter(sensorP1.hgts, sensorP1.hXY, senDF.name + '_P1', senDF.dir + '/', label)
        senPlotP1.padMapping()
    
    if len(sensorP2.hgts):
        label = senDF.name + ' P2'
        senPlotP2 = HeightPlotter(sensorP2.hgts, sensorP2.hXY, senDF.name + '_P2', senDF.dir + '/', label)
        senPlotP2.padMapping()

if pcb.exists:
    if len(pcbP1.hgts):
        label = pcbDF.name + ' P1'
        pcbPlotP1 = HeightPlotter(pcbP1.hgts, pcbP1.hXY, pcbDF.name + '_P1', pcbDF.dir + '/', label)
        pcbPlotP1.padMapping()
        
    if len(pcbP2.hgts):
        label = pcbDF.name + ' P2'
        pcbPlotP2 = HeightPlotter(pcbP2.hgts, pcbP2.hXY, pcbDF.name + '_P2', pcbDF.dir + '/', label)
        pcbPlotP2.padMapping()
        
if survey.exists:
    if len(surveyComp.hgts):
        label = surveyDF.name + ' Survey'
        surveyPlot = HeightPlotter(surveyComp.hgts, surveyComp.hXY, surveyDF.name + '_survey', surveyDF.dir + '/', label)
        surveyPlot.padMapping()